In [1]:
# Basics
import numpy as np
import pandas as pd
import os
import pickle
import h5py

# Keras
from keras.optimizers import Adam, Nadam
from keras.callbacks import ModelCheckpoint, TensorBoard

/Users/anders1991/miniconda3/envs/py3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
% cd /Users/anders1991/deepbreath/

/Users/anders1991/deepbreath


# Set hyperparameters

In [3]:
name = "test"
classification = False
timesteps = 1
batch_size = 5
learn_rate = 1e-4
max_epochs = 50
downsample = 1
droprate = 0.5
overfit = True

if classification:
    loss = "categorical_crossentropy"
else:
    loss = "mean_absolute_error"

if learn_rate > 0:
    optimizer = Adam(lr = learn_rate)
    opt = "Adam"
else:
    optimizer = Nadam()
    opt = "Nadam"

# Load data partitions

In [4]:
with open("./data/partition.pkl", 'rb') as f:
    partition = pickle.load(f)
    
if overfit:
    partition["train"] = partition["train"][:20]
    partition["valid"] = partition["valid"][:20]

# Load labels

In [5]:
target = pd.read_csv("./data/ERU_Scores_Ids_5-Scans_Validity-0_VisuallyScored.csv")
labels = target.set_index("StId").to_dict()["ERU.M2"]

In [6]:
# Rescale labels
if classification:
    # combine 0+1 as 0 = no emph in scan, 1 = no emph in region
    label_converter = {0: 0, 1: 0, 2: 2, 3: 3, 4: 4, 5: 5, 6: 6}
else:
    # Rescale labels;
    label_converter = {0: 0.0, 1: 0.0, 2: 0.03, 3: 0.155, 4: 0.38, 5: 0.63, 6: 0.88}
labels = {key: label_converter[val] for key, val in labels.items()}

# Calculate class weights

To handle imbalanced classes during training.

In [7]:
from sklearn.utils import class_weight

In [8]:
# weights only based on training data
train_labels = [label for key, label in labels.items() if key in partition["train"]]

In [9]:
class_weights = class_weight.compute_class_weight(class_weight='balanced',
                                                  classes=np.unique(train_labels), 
                                                  y=train_labels)

# Import data generator

In [10]:
from scripts.data_gen import DataGenerator

In [11]:
trainGen = DataGenerator("train", classification=classification, batch_size=batch_size,
                         timesteps=timesteps, channels=1, dim_x=142, dim_y=322, dim_z=262, shuffle=True)
validGen = DataGenerator("valid", classification=classification, batch_size=batch_size,
                         timesteps=timesteps, channels=1, dim_x=142, dim_y=322, dim_z=262, shuffle=True)

trainGen = trainGen.generate(labels, partition["train"])
validGen = validGen.generate(labels, partition["valid"])

# Import model

In [12]:
from scripts.unet import tdist_unet

In [13]:
model = tdist_unet(classification=classification, timesteps=timesteps, downsample=downsample, droprate=0.5)

In [14]:
model.compile(optimizer=optimizer, 
              loss=loss, 
              metrics=['accuracy', 'mae', 'mse'])

In [15]:
model.summary(line_length=130)

__________________________________________________________________________________________________________________________________
Layer (type)                              Output Shape                 Param #         Connected to                               
input_1 (InputLayer)                      (None, 1, 1, 108, 284, 228)  0                                                          
__________________________________________________________________________________________________________________________________
TD_mpool3D (TimeDistributed)              (None, 1, 1, 108, 284, 228)  0               input_1[0][0]                              
__________________________________________________________________________________________________________________________________
TD_conv_1 (TimeDistributed)               (None, 1, 6, 108, 280, 224)  156             TD_mpool3D[0][0]                           
___________________________________________________________________________________

# Setup training session

In [15]:
# directory = "./output/"+name+"/"
# if not os.path.exists(directory):
#     os.makedirs(directory)

# with open("./output/"+name+"/config.txt", "w") as txt:
#     txt.write("name = {0}\n".format(name))
#     txt.write("timesteps = {0}\n".format(timesteps))
#     txt.write("batch_size = {0}\n".format(batch_size))
#     txt.write("learn_rate = {0}\n".format(learn_rate))
#     txt.write("max_epochs= {0}\n".format(max_epochs))
#     txt.write("downsample = {0}\n".format(downsample))
#     txt.write("droprate = {0}\n".format(droprate))
#     txt.write("mode = {0}\n".format(mode))
#     txt.write("loss = {0}\n".format(loss))
#     txt.write("opt = {0}\n".format(opt))

# callbacks_list = []

Save model weights each epoch if best val acc so far

In [16]:
# modeldir = directory + "epoch_{epoch:02d}-valacc_{val_acc:.2f}.hdf5"
# checkpoint = ModelCheckpoint(modeldir, monitor='val_acc', save_weights_only=False, save_best_only=True, mode='max', verbose=1)
# callbacks_list.append(checkpoint)

Add TensorBoard callback

In [17]:
# from TBCallbacks import TrainValTensorBoard

# tensorboard= TrainValTensorBoard(log_dir="./output/"+name+"/logs", histogram_freq=0, write_graph=True, write_images=True) # custom TB writer object
# callbacks_list.append(tensorboard) # add tensorboard logging

# Train model

In [18]:
# hist = model.fit_generator(generator = trainGen,
#                            steps_per_epoch = len(partition["train"])//batch_size,
#                            validation_data = validGen,
#                            validation_steps = len(partition["valid"])//batch_size,
#                            class_weight = class_weights,
#                            epochs = max_epochs,
#                            callbacks=callbacks_list)